# Zhang's method

In [ ]:
# import necessary libraries
import os
import cv2
import glob
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
path = "C:/Users/sreed/My_Python_Scripts/Computer Vision/src/images - Assignment1/Zhangs"

zhang_img_1 = './IMG_5456.jpg'
zhang_img_2 = './IMG_5470.jpg'


cbraw = 6#6
cbcol = 8#4
# prepare object points, like (0,0,0), (1,0,0), (2,0,0) ....,(6,5,0)
objp = np.zeros((cbraw*cbcol,3), np.float32) 
 
objp[:,:2] = np.mgrid[0:cbraw,0:cbcol].T.reshape(-1,2)

print(objp.shape)

criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001)

In [24]:

objpoints = [] # 3d point in real world space
imgpoints = [] # 2d points in image plane.
images = os.listdir(path)
print(images)
for fname in images:
# For each picture, identify the corner points, record the world object coordinates and image coordinates
    
    img = cv2.imread(fname) #source image
    #The picture I used is too big and it is reduced by half
    img = cv2.resize(img,None,fx=0.5, fy=0.5, interpolation = cv2.INTER_CUBIC)
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY) #Turn to grayscale
   
    #Find corner points, save in corners, ret is the flag to find corner points
    ret, corners = cv2.findChessboardCorners(gray,(6,8),None)
    
    corners2 = cv2.cornerSubPix(gray,corners,(11,11),(-1,-1),criteria)

    objpoints.append(objp)
    imgpoints.append(corners2)
    
    #Draw corner points on the board, just a visualization tool
    img = cv2.drawChessboardCorners(gray,(6,8),corners2,ret)
    img = cv2.resize(img, (5472, 3648)) 
    cv2.imwrite("{0}_corners.jpg".format(fname.rsplit('.', 1)[0]),img)
    cv2.waitKey(10000)

    ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, gray.shape[::-1],None,None)
    img = cv2.imread("{0}_corners.jpg".format(fname.rsplit('.', 1)[0]))
    
    img = cv2.resize(img,None,fx=0.8, fy=0.8, interpolation = cv2.INTER_CUBIC)
    h,w = img.shape[:2]

    newcameramtx, roi=cv2.getOptimalNewCameraMatrix(mtx,dist,(w,h),1,(w,h))
    
    #Correct distortion
    dst = cv2.undistort(img, mtx, dist, None, newcameramtx)
    
    print('Rotation vector')
    print(rvecs)
    print('Translation vector')
    print(tvecs)
    
    #Output the picture after correcting the distortion
    x,y,w,h = roi
    dst = dst[y:y+h, x:x+w]
    cv2.imwrite('calibresult_{0}.jpg'.format(fname.rsplit('.', 1)[0]),dst)
    
    #Print the two matrix parameters
    print("newcameramtx:\n",newcameramtx)
    print("dist:\n",dist)
    
    #Calculation error
    tot_error = 0
    for i in range(len(objpoints)):
        imgpoints2, _ = cv2.projectPoints(objpoints[i], rvecs[i], tvecs[i], mtx, dist)
        error = cv2.norm(imgpoints[i],imgpoints2, cv2.NORM_L2)/len(imgpoints2)
        tot_error += error

    print("total error: ", tot_error/len(objpoints))


(48, 3)
['IMG_5456.JPG', 'IMG_5470.jpg']
newcameramtx:
 [[3.02642227e+04 0.00000000e+00 1.91526977e+03]
 [0.00000000e+00 2.98069062e+04 7.55451638e+02]
 [0.00000000e+00 0.00000000e+00 1.00000000e+00]]
dist:
 [[ 5.20552114e+00 -2.08498934e+02 -4.07992380e-02  7.84121743e-02
  -3.65848530e-01]]
total error:  0.053613417210634035
newcameramtx:
 [[6.18873633e+03 0.00000000e+00 9.15664077e+02]
 [0.00000000e+00 5.91365381e+03 1.20269056e+03]
 [0.00000000e+00 0.00000000e+00 1.00000000e+00]]
dist:
 [[ 1.37124823e-01  2.12906469e+00  8.06192962e-03 -1.80442793e-02
  -2.49184752e+01]]
total error:  0.11544347570472387


#  Overlay a wireframe

In [32]:

img = cv2.imread("IMG_5456.jpg")
overlay_t = cv2.imread("IMG_5456_corners.jpg") # -1 loads with transparency
height1, width1 = img.shape[:2]
height2, width2 = overlay_t.shape[:2]
def overlay_transparent(bg_img, img_to_overlay_t):
    
    # Extract the alpha mask of the RGBA image, convert to RGB 
    b,g,r = cv2.split(img_to_overlay_t)
    overlay_color = cv2.merge((b,g,r))

    mask = cv2.medianBlur(a,5)

    # Black-out the area behind the logo in our original ROI
    img1_bg = cv2.bitwise_and(bg_img.copy(),bg_img.copy(),mask = cv2.bitwise_not(mask))

    # Mask out the logo from the logo image.
    img2_fg = cv2.bitwise_and(overlay_color,overlay_color,mask = mask)

    # Update the original image with our new ROI
    bg_img = cv2.add(img1_bg, img2_fg)
    cv2.imwrite('overlay.jpg',bg_img)
    print(bg_img.shape[:2])
    return bg_img

bg = overlay_transparent(img, overlay_t)
print(bg.shape[:2])
cv2.imshow('image',bg)
cv2.waitKey(1000)

(3648, 5472)
(3648, 5472)


-1